In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import re
import pickle
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import porter
import pymongo
from pymongo import MongoClient

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.decomposition import TruncatedSVD


from sklearn.decomposition import NMF

from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans



In [2]:
client = MongoClient()
db = client.jarvis
#collection = db.news1
#data = pd.DataFrame(list(collection.find()))

In [3]:
#text=doc['Cleanedcontent']
jarviscursor = db.cleanedcontent.find()[0:2]
for doc in jarviscursor:
    print(str(doc['Cleanedcontent']),'\n')

['washington', 'congression', 'republican', 'new', 'fear', 'health', 'lawsuit', 'obama', 'administr', 'might', 'win', 'incom', 'trump', 'administr', 'could', 'choos', 'longer', 'defend', 'execut', 'branch', 'suit', 'challeng', 'administr', 'author', 'spend', 'billion', 'dollar', 'health', 'insur', 'subsidi', 'american', 'hand', 'hous', 'republican', 'big', 'victori', 'issu', 'sudden', 'loss', 'disput', 'subsidi', 'could', 'conceiv', 'caus', 'health', 'program', 'implod', 'leav', 'million', 'peopl', 'without', 'access', 'health', 'insur', 'befor', 'republican', 'prepar', 'replac', 'could', 'lead', 'chao', 'insur', 'market', 'spur', 'polit', 'backlash', 'republican', 'gain', 'full', 'control', 'govern', 'stave', 'outcom', 'republican', 'could', 'find', 'themselv', 'awkward', 'posit', 'appropri', 'huge', 'sum', 'temporarili', 'prop', 'obama', 'health', 'law', 'anger', 'conserv', 'voter', 'demand', 'law', 'year', 'anoth', 'twist', 'donald', 'trump', 'administr', 'worri', 'preserv', 'execut

In [13]:
gen = (doc['Cleanedcontent'] for doc in db.cleanedcontent.find()[0:2])
for file in gen:
    print(file,'\n')

['washington', 'congression', 'republican', 'new', 'fear', 'health', 'lawsuit', 'obama', 'administr', 'might', 'win', 'incom', 'trump', 'administr', 'could', 'choos', 'longer', 'defend', 'execut', 'branch', 'suit', 'challeng', 'administr', 'author', 'spend', 'billion', 'dollar', 'health', 'insur', 'subsidi', 'american', 'hand', 'hous', 'republican', 'big', 'victori', 'issu', 'sudden', 'loss', 'disput', 'subsidi', 'could', 'conceiv', 'caus', 'health', 'program', 'implod', 'leav', 'million', 'peopl', 'without', 'access', 'health', 'insur', 'befor', 'republican', 'prepar', 'replac', 'could', 'lead', 'chao', 'insur', 'market', 'spur', 'polit', 'backlash', 'republican', 'gain', 'full', 'control', 'govern', 'stave', 'outcom', 'republican', 'could', 'find', 'themselv', 'awkward', 'posit', 'appropri', 'huge', 'sum', 'temporarili', 'prop', 'obama', 'health', 'law', 'anger', 'conserv', 'voter', 'demand', 'law', 'year', 'anoth', 'twist', 'donald', 'trump', 'administr', 'worri', 'preserv', 'execut

In [4]:
cv = CountVectorizer(lowercase=False, input='content', stop_words='english')
gen = (doc['Cleanedcontent'] for doc in db.cleanedcontent.find())
cv.fit(gen)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [24]:
lda = LatentDirichletAllocation(n_components=50)

In [6]:
gen = (doc['Cleanedcontent'] for doc in db.cleanedcontent.find().noCursorTimeout())
#for epoch in range(10):
    for file in gen:
        vec_file = cv.transform([file])
        lda.partial_fit(vec_file)


AttributeError: 'Cursor' object has no attribute 'noCursorTimeout'

In [7]:
gen = (doc['Cleanedcontent'] for doc in db.cleanedcontent.find({},no_cursor_timeout=True))
for epoch in range(1):
    for file in gen:
        vec_file = cv.transform([file])
        lda.partial_fit(vec_file)


In [25]:
a = db.cleanedcontent.aggregate( [ { '$sample': {'size' : 1000}}])
#for doc in list(a):
 #   print(doc['Cleanedcontent'],'\n')

In [26]:

for epoch in range(10):
    gen = [doc['Cleanedcontent'] for doc in list(a)]
    for file in gen:
        vec_file = cv.transform([file])
        lda.partial_fit(vec_file)


In [22]:
# for 1000 docs, 10 epochs
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [16]:
#for 100 docs
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
display_topics(lda, cv.get_feature_names(), 10)


Topic  0
said, offici, russia, russian, secur, presid, compani, fbi, unit, foreign

Topic  1
trump, hi, republican, presid, campaign, democrat, elect, donald, obama, vote

Topic  2
post, facebook, twitter, photo, video, caption, new, stori, okay, restaur

Topic  3
percent, research, rate, year, peopl, use, say, like, make, increas

Topic  4
said, state, islam, wa, attack, militari, forc, countri, unit, syria

Topic  5
wa, hi, like, peopl, say, think, know, make, time, don

Topic  6
said, wa, polic, offic, kill, shoot, gun, told, peopl, accord

Topic  7
clinton, wa, said, hi, state, law, court, report, case, news

Topic  8
year, wa, new, american, world, time, washington, group, compani, nation

Topic  9
said, game, tax, million, year, plan, insur, money, billion, bank


In [28]:
# for 1000 docs
display_topics(lda, cv.get_feature_names(), 10)


Topic  0
american, nation, speech, billion, histori, deal, valu, gain, base, score

Topic  1
member, world, human, leader, gay, cultur, intern, critic, hotel, institut

Topic  2
wa, peopl, like, make, new, work, think, mani, onli, said

Topic  3
trump, clinton, republican, donald, democrat, elect, hillari, parti, campaign, polit

Topic  4
april, record, perform, movi, island, artist, type, music, struck, success

Topic  5
million, school, student, children, need, parent, say, colleg, famili, mani

Topic  6
black, claim, white, power, racial, matter, distribut, year, old, brown

Topic  7
hi, game, compani, support, open, appear, seri, fan, star, final

Topic  8
oper, militari, europ, warn, foreign, germani, air, european, minist, attempt

Topic  9
law, court, govern, feder, abort, suprem, depart, group, local, educ

Topic  10
post, person, left, media, question, wrote, write, newspap, magazin, editor

Topic  11
china, industri, technolog, firm, agreement, meet, labor, analyst, econom, 

In [12]:
lsa = TruncatedSVD(2, algorithm = 'randomized')
a = db.cleanedcontent.aggregate( [ { '$sample': {'size' : 1000}}])

for epoch in range(1):
    gen = [doc['Cleanedcontent'] for doc in list(a)]
    for file in gen:
        vec_file = cv.transform([file])
        dtm_lsa = lsa.fit_transform(vec_file)
        dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)


# Fit LSA. Use algorithm = “randomized” for large datasets 
#lsa = TruncatedSVD(2, algorithm = 'arpack')

#dtm_lsa = lsa.fit_transform(dtm)
#dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)



/home/sangrador/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/truncated_svd.py:192: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [13]:
display_topics(lsa, cv.get_feature_names(), 10)


Topic  0
medicaid, drug, state, wisconsin, said, program, new, health, walker, abus


In [14]:
nmf_model = NMF(2)

a = db.cleanedcontent.aggregate( [ { '$sample': {'size' : 1000}}])
for epoch in range(1):
    gen = [doc['Cleanedcontent'] for doc in list(a)]
    for file in gen:
        vec_file = cv.transform([file])
        dtm_nmf = nmf_model.fit_transform(vec_file)
        dtm_nmf = Normalizer(copy=False).fit_transform(dtm_nmf)






IndexError: index 1 is out of bounds for axis 1 with size 1

In [10]:
display_topics(lda, cv.get_feature_names(), 10)


Topic  0
year, wa, mani, think, women, world, american, time, peopl, new

Topic  1
republican, vote, state, democrat, voter, parti, elect, support, poll, campaign

Topic  2
trump, hi, said, presid, wa, donald, obama, white, elect, hillari

Topic  3
said, wa, court, law, case, state, depart, offic, school, justic

Topic  4
percent, compani, year, new, research, rate, market, said, use, data

Topic  5
hi, wa, game, play, team, year, night, star, season, best

Topic  6
clinton, campaign, post, report, senat, ani, group, wa, support, day

Topic  7
state, said, unit, russia, russian, govern, countri, north, offici, presid

Topic  8
wa, say, like, peopl, know, don, becaus, thing, famili, day

Topic  9
said, wa, polic, hi, state, kill, offic, attack, forc, islam
